<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Getting Started With Spark using Python**


Estimated time needed: **15** minutes


![](http://spark.apache.org/images/spark-logo.png)


### The Python API


Spark is written in scala, which compiles to java bytecode, but you can write python code to communicate to the java virtual machine through a library called py4j.  Python has the richest API, but it can be somewhat limiting if you need to use a method that is not available, or if you need to write a specialized piece of code.  The latency associated with communicating back and forth to the JVM can sometimes cause the code to run slower.

An exception to this is the SparkSQL library, which has an execution planning engine that precompiles the queries.  Even with this optimization, there are cases where the code may run slower than the native scala version.

The general recommendation for pyspark code is to use the "out of the box" methods available as much as possible, and avoid overly frequent (iterative) calls to Spark methods.  If you need to write really high performance or specialized code, try doing it in scala.

But hey, we know.  Python rules, and the plotting libraries are way better.  So, it's up to you, really!


## Objectives


In this lab, we will go over the basics of Apache Spark and pyspark. We will start with creating the SparkContext and SparkSession. We then create an RDD and apply some basic transformations and actions. Finally we demonstrate the basics dataframes and SparkSQL.

*   Spark Context and Spark Session
*   RDDs: Creation, Transofrmations, Actions and Caching.
*   Creating a DataFrame
*   SparkSQL basics


***


## Setup


For this lab, we are going to be using Python and Spark (pyspark). These libraries should be installed in your lab environment or in SN Labs.


In [ ]:
# How to download data using a URL
#!wget https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DL0110EN/edx_project/circle.zip

In [ ]:
# Unzipping a folder
# !unzip circle.zip
# It is a good idea to apply the -o and -q  when unzipping to quiet the process and overwrite any existing folders
# !unzip -o -q circle.zip

In [1]:
# pyspark is the Spark API for Python. In this lab, we use pyspark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## Exercise 1 -  Spark Context and Spark Session


In this exercise, you will create the Spark Context and initialize the Spark session needed for SparkSQL and DataFrames.
SparkContext is the entry point for Spark applications and contains functions to create RDDs such as `parallelize()`. SparkSession is needed for SparkSQL and DataFrame operations.


#### Task 1: Creating the spark session and context


In [2]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#### Task 2: Initialize Spark session

To work with dataframes we just need to verify that the spark session instance has been created.
Feel free to click on the "Spark UI" button to explore the Spark UI elements.


In [3]:
spark

## Exercise 2: RDDs

In this exercise we work with Resilient Distributed Datasets (RDDs). RDDs are Spark's primitive data abstraction and we use concepts from functional programming to create and manipulate RDDs.


#### Task 1: Creating an RDD.

For demonstration purposes, we create an RDD here by calling `sc.parallelize()`\
We create an RDD which has integers from 1 to 30.


In [4]:
data = range(1,30)
# print first element of iterator
print(data[0])
len(data)
xrangeRDD = sc.parallelize(data, 4)

# this will let us know that we created an RDD
xrangeRDD

1


PythonRDD[1] at RDD at PythonRDD.scala:53

#### Task 2: Transformations


A transformation is an operation on an RDD that results in a new RDD.  The transformed RDD is generated very rapidly, because the new RDD is *lazily evaluated*, which means that the calculation is not carried out when the new RDD is generated.  The RDD will contain a series of transformations, or computation instructions, that will only be carried out when an action is called. In this transofrmation, we reduce each element in the RDD by 1. Note the use of the `lambda` function. We also then filter the RDD to only contain elements <10.


In [5]:
subRDD = xrangeRDD.map(lambda x: x-1)
filteredRDD = subRDD.filter(lambda x : x<10)


#### Task 3: Actions


A transformation returns a result to the driver. We now apply the `collect()` action to get the output from the transformation.


In [6]:
print(filteredRDD.collect())
filteredRDD.count()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


10

#### Task 4: Caching Data


This simple example shows how to create an RDD and cache it. Notice the **10x speed improvement**!  If you wish to see the actual computation time, browse to the Spark UI...it's at host:4040.  You'll see that the second calculation took much less time!


In [7]:
import time 

test = sc.parallelize(range(1,50000),4)
test.cache()

t1 = time.time()
# first count will trigger evaluation of count *and* cache
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)


t2 = time.time()
# second count operates on cached data only
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)

#test.count()

dt1:  0.2962369918823242
dt2:  0.04923295974731445


## Exercise 3: DataFrames and SparkSQL


In order to work with the extremely powerful SQL engine in Apache Spark, you will need a Spark Session. We have created that in the first Exercise, let us verify that spark session is still active.


In [8]:
spark

#### Task 1: Create Your First DataFrame!


You can create a structured data set (much like a database table) in Spark.  Once you have done that, you can then use powerful SQL tools to query and join your dataframes.


In [9]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/people.json >> people.json

df = spark.read.json("people.json").cache()
df.show()
df.printSchema()

# Register the DataFrame as a SQL temporary view
df.createTempView("people")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    73  100    73    0     0    366      0 --:--:-- --:--:-- --:--:--   366
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|null|Michael|
|  30|   Andy|
|  19| Justin|
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



#### Task 2: Explore the data using DataFrame functions and SparkSQL

In this section, we explore the datasets using functions both from dataframes as well as corresponding SQL queries using sparksql. Note the different ways to achieve the same task!


In [10]:
# Select and show basic data columns

df.select("name").show()
df.select(df["name"]).show()
spark.sql("SELECT name FROM people").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
+-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
+-------+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
|Michael|
|   Andy|
| Justin|
+-------+



In [11]:
# Perform basic filtering

df.filter(df["age"] > 21).show()
spark.sql("SELECT age, name FROM people WHERE age > 21").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
| 30|Andy|
| 30|Andy|
+---+----+

+---+----+
|age|name|
+---+----+
| 30|Andy|
| 30|Andy|
| 30|Andy|
+---+----+



In [12]:
# Perfom basic aggregation of data

df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

+----+-----+
| age|count|
+----+-----+
|  19|    3|
|null|    3|
|  30|    3|
+----+-----+

+----+-----+
| age|count|
+----+-----+
|  19|    3|
|null|    0|
|  30|    3|
+----+-----+



***


### Question 1 - Type in your question here


Content of Question 1


In [13]:
# Code block for Question 1

In [14]:
# Code block for learners to answer

### Question 2 - Type in your question here


Here is a sample question, using the following values please use the plotter helper function to plot the data. Please type your answer in the available code box below.


In [15]:
# This function adds the two parameters given a and b
x = [0, 1, 2, 3, 4]
y = [0, 1, 4, 9, 16]
title = 'y = x^2'

In [16]:
# Code block for learners to answer

### Question 3 - Type in your question here


Content for Question 3


In [17]:
# Code block for Question 3

In [18]:
# Code block for learners to answer

Double-click **here** for a hint.

<!-- The hint is below:

Add the hint here

-->


Double-click **here** for the solution.

<!-- The answer is below:

Add the solution here

-->


## Authors


[Author1 Name (with link to social profile)](author_link)


### Other Contributors


[Contributor with Link](contributor_linl), Contributor No Link


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2020-07-20        | 0.2     | Azim       | Modified Multiple Areas |
| 2020-07-17        | 0.1     | Azim       | Created Lab Template    |


Copyright © 2021 IBM Corporation. All rights reserved.
